In [31]:
# import necessary libraries
from mnist import MNIST
import random
import numpy as np
import time
import time
import pickle

In [32]:
# define Kernel density estimation using gaussian kernel
def Gausskernel(x_a,x_b,sigma):
    len_train = len(x_a)
    len_val = len(x_b)
    d = len(x_a[0].astype(np.float64))
    mu = x_a.astype(np.float64)
    # the term log(1/K) where k is all the elements in the set
    ln_k = np.log(len_train)
    #denominator 2σ^2
    denom = 2*(sigma**2)
    # the term 0.5(log(2πσ^2) is constant
    t = -(0.5*d)*np.log((np.pi*denom))
    p_x = 0
    for i in range(len_train):
        x = x_b[i].astype(np.float64)
        t1 = np.sum((-(x-mu)**2)/denom, axis=1)
        a = np.max(t1)
        sum_ele = np.sum(np.exp(t1 - a))
        log_p = a + np.log(sum_ele)
        p_x += log_p
    f = t - ln_k +p_x / len_val
    return f

In [33]:
#Load MNIST Data
mndata = MNIST('.\data\MNIST')
mndata.gz = True
images_train, labels_train = mndata.load_training()
# or
images_test, labels_test = mndata.load_testing()
images_train = np.array(images_train)
images_test = np.array(images_test)


In [34]:
#Print random data 
index = random.randrange(0, len(images_train))  # choose an index ;-)
print(mndata.display(images_train[index]))


............................
............................
............................
............................
................@@@@........
................@@@@........
................@@@@........
...............@@@@@........
...............@@@@.........
...............@@@@.........
..............@@@@@.........
..............@@@@@.........
..............@@@@@.........
.............@@@@@@.........
.............@@@@@..........
............@@@@@...........
...........@@@@@............
...........@@@@@............
...........@@@@@............
..........@@@@@.............
.........@@@@@..............
........@@@@@@..............
........@@@@@...............
........@@@@@............@..
............................
............................
............................
............................


In [35]:
#Carve out train, validationa nd test sets and preprocess by scaling values
np.random.shuffle(images_train)

X_train = images_train[0:10000]
X_valid = images_train[10000:20000]
X_test = images_test

X_train = np.array(X_train)
X_valid = np.array(X_valid)
X_test = np.array(X_test)

X_train = X_train/255
X_valid = X_valid/255
X_test = X_test/255

print(np.max(X_train))
print(np.min(X_train))


1.0
0.0


In [36]:
# Grid-search for the optimal value of sigma, returning the log-probability of each sigma
sigmas = [0.05, 0.08, 0.1, 0.2, 0.5, 1.0, 1.5, 2.0 ]
p = [kernel(X_train[0:100],X_valid[0:100],sigma) for sigma in sigma_range]
print (p)
indx = np.argmax(p)
# log-probability for optimal sigma and calculate time
start_time = time.time()
sigmax = Gausskernel(X_train[0:100],X_test[0:100],sigmas[indx])
print("--- %s seconds ---" % (time.time() - start_time))
print (sigmax)

[-7900.898101127743, -2465.3892320360337, -1300.951063591691, -58.53131190753254, -276.79438448310384, -748.6145108829477, -1052.944381440686, -1273.3483799745331]
--- 0.0280001163482666 seconds ---
-77.21941571962122


In [37]:
#Load CIFAR_100 Data and carve out train, validation and test sets and scale values (pre-process)

def unpickle(file):
    fo = open(file, 'rb')
    u = pickle._Unpickler( fo )
    u.encoding = 'latin1'
    dict = u.load()
    fo.close()
    return dict

def load_files(train_dir, test_dir):
    Train = unpickle(train_dir)
    Test = unpickle(test_dir)
    X_training =Train['data'].astype(np.float64)
    X_training = X_training/255
    np.random.shuffle(X_training)
    X_train = X_training[0:10000]
    X_valid = X_training[10000:20000]
    X_test =Test['data'].astype(np.float64)
    X_test = X_test/255
    return X_train, X_valid, X_test

X_train_c, X_valid_c, X_test_c = load_files('./data/cifar-100-python/train', './data/cifar-100-python/test') 

In [38]:
# Grid-search for the optimal value of sigma, returning the log-probability of each sigma
sigmas = [0.05, 0.08, 0.1, 0.2, 0.5, 1.0, 1.5, 2.0]
p = [kernel(X_train_c[0:10000],X_valid_c[0:10000],sigma) for sigma in sigmas]
print (p)
indx = np.argmax(p)
start_time = time.time()
# log-probability for optimal sigma and calculate time
sigmax = Gausskernel(X_train_c[0:10000],X_test_c[0:10000],sigmas[indx])
print (sigmax)
print("--- %s seconds ---" % (time.time() - start_time))

[-22103.404372850302, -6193.041706017947, -2873.7194890367828, 336.6969672859834, -982.9962640304133, -2898.6909295429705, -4104.632894519445, -4974.396457961507]
70.6630225740596
--- 0.7890000343322754 seconds ---
